In [2]:
import numpy as np
import pandas as pd
from mlfinlab.online_portfolio_selection import *
#from mlfinlab.online_portfolio_selection.BAH import BAH


/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)




Support us on Patreon: https://www.patreon.com/HudsonThames

MlFinLab needs you! We need your help for us to keep on maintaining and implementing academic research based on 
financial machine learning (for open-source). In order for us to continue we need to raise $4000 of monthly donations
via Patreon - by December 2020. If we can't reach our goal, we will need to adopt more of a paid for service. We thought
that the best and least impactful course of action (should we not reach our goal) is to leave the package as open-source
but to make the documentation (ReadTheDocs) a paid for service. This is the ultimate litmus test, if the package is a 
value add, then we need the community to help us keep it going.

Our road map for 2020 is to implement the text book: Machine Learning for Asset Managers by Marcos Lopez de Prado, 
as well as a few papers from the Journal of Financial Data Science. We are hiring a full time developer for 3 months 
to help us reach our goals. The money that you

In [3]:
stock_prices = pd.read_csv('../tests/test_data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)
asset_name = list(stock_prices.columns)

In [4]:
idx = stock_prices.index
make_array = np.array(stock_prices)
make_array

array([[ 49.2733345 ,  35.38999939, 106.63999939, ...,  77.36000061,
        101.40000153, 130.63000488],
       [ 49.71666718,  35.29000092, 107.        , ...,  77.45999908,
        101.51999664, 130.74000549],
       [ 48.22333145,  34.59999847, 106.97000122, ...,  77.55000305,
        101.65000153, 128.16999817],
       ...,
       [ 34.02000046,  23.70999908, 116.44999695, ...,  84.19000244,
        105.94000244, 176.63999939],
       [ 34.36000061,  24.04999924, 116.66999817, ...,  84.30000305,
        106.08999634, 179.        ],
       [ 34.68999863,  24.12000084, 117.05999756, ...,  84.37000275,
        106.        , 179.30999756]])

In [16]:
asset = np.array([[30,30,30],[60,60,15],[120,15,60],[60,60,90]])
asset

array([[ 30,  30,  30],
       [ 60,  60,  15],
       [120,  15,  60],
       [ 60,  60,  90]])

In [20]:
asset_change = np.array(pd.DataFrame(asset).pct_change().fillna(0))
asset_change += 1
asset_change

array([[1.  , 1.  , 1.  ],
       [2.  , 2.  , 0.5 ],
       [2.  , 0.25, 4.  ],
       [0.5 , 4.  , 1.5 ]])

In [23]:
w0 = np.array([1/3,1/3,1/3])
r0 = np.dot(w0, asset_change[0])
r0

1.0

In [32]:
w1 = w0
r1 = np.dot(w1, asset_change[1])
r1

1.5

In [33]:
w2 = np.multiply(w1,asset_change[1]) / np.dot(w1, asset_change[1])
r2 = np.dot(w2, asset_change[2])
r2

1.4444444444444444

In [37]:
w3 = np.multiply(w2, asset_change[2]) / np.dot(w2, asset_change[2])
r3 = np.dot(w3, asset_change[3])
r3

1.0769230769230769

In [47]:
new_w = np.array([w1, w2, w3])
new_w = new_w.T
new_w

array([[0.33333333, 0.44444444, 0.61538462],
       [0.33333333, 0.44444444, 0.07692308],
       [0.33333333, 0.11111111, 0.30769231]])

In [50]:
np.prod(np.diagonal(np.dot(asset_change[1:],new_w)))

2.333333333333333